In [10]:
let char_list_to_string cl = String.concat "" (List.map (String.make 1) cl)

val char_list_to_string : char list -> string = <fun>


In [11]:
char_list_to_string [' '; 'a'; ' ';]

- : string = " a "


In [12]:
let string_to_char_list niz = List.init (String.length niz) (String.get niz)

val string_to_char_list : string -> char list = <fun>


In [13]:
string_to_char_list " ab c"

- : char list = [' '; 'a'; 'b'; ' '; 'c']


In [14]:
String.trim "   abc    "

- : string = "abc"


In [16]:
let trak1 = "   abcDe  fg    h    "
(* Imam torej kao glavo na D. in mam en seznam, k je c b a prazen, in morm sam to nakonc porezat, pa obrnt.*)
(* Lahko levga najprej obrnem, potem najdem find_index od prvega elementa, k ni prazen in pol nardim sub
    Desn pa morm najt zadnga, da so od tam naprej sam še presledki ... nočm ga za brez veze obračat ...
    Ker more bit kao učinkovito to vse skupi .... rabla bi sam en left/right trim ...
    Okej pač enkrat se bom mogla čez sprehodt. Grem se probat sam enkrat.
    Za ta desnga. Grem enkrat čez, da dobim indeks tazadnga k je blank z eno funkcijo in pol substring.
    Grem torej dvakrat čez. Kar je bolš, kokr da dvakrat obrnem in enkrat iščem.
    Za ta levga pa pač obrnem in grem do tam, kjer prvič je znak.
    Kaj pa če mam desn seznam in grem čez, da najdem zadnjega praznega
in potem še enkrat čez sam do unga indeksa z nekim mapi in hkrati nardim string.
Pa a se pozna, da nardim tri stringe in jih concateniram skupi, kokr pa da dodajam na enga?
Sej ne vem, kako stringi delujejo - a je sploh razlika?*)

val trak1 : string = "   abcDe  fg    h    "


In [17]:
String.split_on_char ' ' trak1

- : string list =
[""; ""; ""; "abcDe"; ""; "fg"; ""; ""; ""; "h"; ""; ""; ""; ""]


In [18]:
let pomozna_desni1 desni_char_list =
  let rec aux i_konca i carry sez =
    match carry, sez with
    | _, [] -> i_konca (*če je praten, bo 0, bo prazen string nakonc*)
    | true, x :: xs when x = ' ' -> aux i_konca (i + 1) true xs (* vrsta presledkov *)
    | false, x :: xs when x = ' ' -> aux i (i + 1) true xs (*vidu prvi presledek po drugih znakih*)
    | carry, x :: xs -> aux (i + 1) (i + 1) false xs (*kerkol drug znak pa izklop zastavco, pa ta znak mora bit v nizu, tkoda i + 1
    ker bo šlo do ne-vključno. Torej raj nardim, da gre do vključno, in se poveča vsakič, k je neprazen znak.
    Pa ne rabm zastavce ?*)
  in
  aux 0 0 false desni_char_list

val pomozna_desni1 : char list -> int = <fun>


In [19]:
pomozna_desni1 (string_to_char_list " ab c   ")

- : int = 5


In [20]:
pomozna_desni1 (string_to_char_list " ")

- : int = 0


In [21]:
let pomozna_desni2 desni_char_list =
  let rec aux i_konca i sez =
    match sez with
    | [] -> i_konca (*če je praten, bo 0, bo prazen string nakonc*)
    | x :: xs when x = ' ' -> aux i_konca (i + 1) xs (* vrsta presledkov *)
    | x :: xs -> aux i (i + 1) xs (*kerkol drug znak pa izklop zastavco, pa ta znak mora bit v nizu, tkoda i + 1
    ker bo šlo do ne-vključno. Torej raj nardim, da gre do vključno, in se poveča vsakič, k je neprazen znak.
    Pa ne rabm zastavce ?*)
  in
  aux 0 0 desni_char_list

val pomozna_desni2 : char list -> int = <fun>


In [22]:
pomozna_desni2 (string_to_char_list "     ")

- : int = 0


In [24]:
let pomozna_desni desni_char_list =
  let rec aux_iskanje_konca i_konca i sez =
    match sez with
    | [] -> i_konca (*če je praten, bo 0, bo prazen string nakonc*)
    | x :: xs when x = ' ' -> aux_iskanje_konca i_konca (i + 1) xs
    | x :: xs -> aux_iskanje_konca i (i + 1) xs (*Torej raj nardim, da gre do vključno, in se poveča vsakič, k je neprazen znak.*)
  in
  let indeks_konca_niza = aux_iskanje_konca 0 0 desni_char_list in
  let zberi_začetek_v_niz i_konca cl =
    let rec aux_v_niz acc i l =
      match i, l with
      | _, [] -> acc
      | i, x :: xs when i <= i_konca -> aux_v_niz (acc ^ Char.escaped x) (i + 1) xs
      | i, x :: xs -> acc (* Nas ne zanima več*)
      (* to men zgleda zelo ne dobr časovno, ker gre kvadratično čez string
      Sam kako pa list.concat to nardi? ...... ???*)
    in
    aux_v_niz "" 0 cl
  in
  zberi_začetek_v_niz indeks_konca_niza desni_char_list
  (*hotla sm neki z List.mapi in pol vrže ven vse ostale.
  Une spremeni v string, tisto pa vrže ven, kar je odveč. A probam?*)
in
let desni_niz = pomozna_desni desni_char_list in

File "[24]", line 9, characters 6-15:
9 |   let zberi_začetek_v_niz i_konca cl =
          ^^^^^^^^^
Alert deprecated: ISO-Latin1 characters in identifiers


error: compile_error

In [27]:
let filter_mapi f =
  let rec aux acc i =
    function
    | [] -> List.rev acc
    | x :: xs -> (
      if Option.is_none (f i x) then aux acc (i + 1) xs
      else aux (Option.get (f i x) :: acc) (i + 1) xs
    )
  in
  aux [] 0

val filter_mapi : (int -> 'a -> 'b option) -> 'a list -> 'b list = <fun>


In [28]:
(* Probam še z vgrajenimi bolj zale konc*)
let pomozna_desni desni_char_list =
  let rec aux_iskanje_konca i_konca i sez =
    match sez with
    | [] -> i_konca (*če je praten, bo 0, bo prazen string nakonc*)
    | x :: xs when x = ' ' -> aux_iskanje_konca i_konca (i + 1) xs
    | x :: xs -> aux_iskanje_konca i (i + 1) xs (*Torej raj nardim, da gre do vključno, in se poveča vsakič, k je neprazen znak.*)
  in
  let indeks_konca_niza = aux_iskanje_konca 0 0 desni_char_list in
  let f_desni_map_z_indeksom_konca i c =
    if i <= indeks_konca_niza then Some (Char.escaped c)
    else None
  in
  filter_mapi f_desni_map_z_indeksom_konca desni_char_list
  |> String.concat ""
  (* ubistvu bi rabla filter_mapi, ki pa ne obstaja, ampak smo ga enkrat implementiral ...*)
  (*hotla sm neki z List.mapi in pol vrže ven vse ostale.
  Une spremeni v string, tisto pa vrže ven, kar je odveč. A probam?*)

val pomozna_desni : char list -> string = <fun>


In [29]:
let cl1 = [' '; ' '; 'a'; 'b'; ' '; 'c'; ' ']

val cl1 : char list = [' '; ' '; 'a'; 'b'; ' '; 'c'; ' ']


In [30]:
pomozna_desni cl1

- : string = "  ab c"


In [31]:
let rec print_list = function 
[] -> ()
| c::l -> Printf.printf "%c" c ; print_list l

val print_list : char list -> unit = <fun>


In [32]:
print_list [' '; ' '; 'l'; 'a'; ' '; 'r'; 'a']

- : unit = ()


In [33]:
Printf.printf "%c" ' '

- : unit = ()


In [35]:
(* Popravek: že sproti naj printa.
Tkoda za desnega lahko nardim tko, da printa dokler je znak, k ni presledek,
pol si shranjuje ostalo v string, in pol, k je spet znak, če je, sprinta najprej un string, pol pa to.*)
let print_desni desni_char_list =
  let rec aux_printd_in_iskanje_konca acc_niz sez =
    match sez with
    | [] -> () (* Pršli do konca alpa je prazen. In nič ne naprinta, kar je okej, ker bo glavo sprintu tud, če bo prazna*)
    | x :: xs when x = ' ' -> aux_printd_in_iskanje_konca (acc_niz ^ (Char.escaped x)) xs (* Če je presledek, da v pomožni niz*)
    | x :: xs -> (
      Printf.printf "%s" acc_niz; Printf.printf "%c" x ; aux_printd_in_iskanje_konca "" xs (*Torej znak, če smo vmes shranjeval presledke, jih zdele še naprintamo*)
    )
  in
  aux_printd_in_iskanje_konca "" desni_char_list

val print_desni : char list -> unit = <fun>


In [ ]:
let print_levi obrnjen_levi_char_list =
  let rec aux_ltrim_in_printl flag sez =
    match flag, sez with
    | _, [] -> ()
    | false, x :: xs when x = ' ' -> aux_ltrim_in_printl false xs
    | false, x :: xs -> Printf.printf "%c" x ; aux_ltrim_in_printl true xs
    | true, x :: xs -> Printf.printf "%c" x ; aux_ltrim_in_printl true xs (* Ko enkrat naleti na znak, ki ni presledek, sprinta vse naprej*)
  in
  aux_ltrim_in_printl false obrnjen_levi_char_list
  (* sam te ta leve morm tud preštet.*)

val print_levi : char list -> unit = <fun>


In [40]:
Printf.printf "%!"

- : unit = ()


In [ ]:
let print_levi_in_prestej obrnjen_levi_char_list =
  let rec aux_ltrim_in_printl flag dolzina_niza sez =
    match flag, sez with
    | _, [] -> dolzina_niza
    | false, x :: xs when x = ' ' -> aux_ltrim_in_printl false (dolzina_niza + 1) xs
    | false, x :: xs -> Printf.printf "%c" x ; aux_ltrim_in_printl true (dolzina_niza + 1) xs
    | true, x :: xs -> Printf.printf "%c" x ; aux_ltrim_in_printl true (dolzina_niza + 1) xs (* Ko enkrat naleti na znak, ki ni presledek, sprinta vse naprej*)
  in
  aux_ltrim_in_printl false 0 obrnjen_levi_char_list
  (* sam te ta leve morm tud preštet.*)
  (* Ne vrne unita, kar je problem, verjetno. Ker pol bo cel print kr en drek vraču.
  Seprav je to vse brezveze in morm sam it čez seznam dvakrat pač, oziroma še enkrat za reverse, tkoda trikrat ...*)

val print_levi_in_prestej : char list -> int = <fun>


In [45]:
(* Pač a profitiram, če dam sproti jih že v string pa sam sprintam pol? *)
(* sam glede na to, da so immutable, pa da dvomm, da so verižni, ker pol ne bi mogu do indeksa dostopat,
verjetno dela skos nove alpa kej. Razen, če je kokr python seznam.
Jap, concat je O(n), tkoda če ga delam na vsakem koraku mam O(n^2)*)
let obrni_ltrim_prestej_levi1 levi_char_list = 
  let rec aux_ltrim_in_prestej i flag sez =
    match flag, sez with
    | _, [] -> i
    | false, x :: xs when x = ' ' -> aux_ltrim_in_prestej 0 false xs
    | false, x :: xs -> aux_ltrim_in_prestej (i + 1) true xs
    | true, x :: xs -> aux_ltrim_in_prestej (i + 1) true xs
  in
  List.rev levi_char_list 
  |> aux_ltrim_in_prestej 0 false
  (* sam js zdej dobim sam int ven, rada bi še nov seznam. sam pol ga morm spet okrog obračat.
  To je še en O(n) in pol še enkrat za print. Ampak sej, it trikrat skoz, manj kot to ne znam, ane?
  Kar je štirkrat vse skupi ...
  Manj lahko dobim: če sproti printam, sam pol nimam informacije, kam dat glavo, morm še enkrat čez itak.
  Kar je tko, kar je zdej, al kaj? In pol je tud čist vseen, a vržem ven seznam takoj, k je neprazen znak,
  pa dam list.length, al jih še js sprot preštejem. In dam v krajši seznam. Al kaj.*)

val obrni_ltrim_prestej_levi : char list -> int = <fun>


In [ ]:
(* Je pa tole cist zabavno drgač :P 21/1/25, 12.31*)

In [55]:
let obrni_ltrim_prestej_levi levi_char_list = 
  let rec aux_ltrim_in_prestej i flag acc sez =
    match flag, sez with
    | _, [] -> i, List.rev acc
    | false, x :: xs when x = ' ' -> aux_ltrim_in_prestej 0 false [] xs
    | false, x :: xs -> aux_ltrim_in_prestej (i + 1) true (x :: acc) xs
    | true, x :: xs -> aux_ltrim_in_prestej (i + 1) true (x :: acc) xs
  in
  List.rev levi_char_list 
  |> aux_ltrim_in_prestej 0 false []
  (* n + n + n *)

val obrni_ltrim_prestej_levi : char list -> int * char list = <fun>


In [47]:
obrni_ltrim_prestej_levi cl1

- : int * char list = (6, ['c'; ' '; 'b'; 'a'; ' '; ' '])


In [48]:
let cl0 = List.init 10 (fun x -> ' ')

val cl0 : char list = [' '; ' '; ' '; ' '; ' '; ' '; ' '; ' '; ' '; ' ']


In [51]:
obrni_ltrim_prestej_levi [' '; ' '; 'l'; ' '; ' '; ' '; ' '; ' '; ' '; ' ']

- : int * char list = (3, ['l'; ' '; ' '])


In [56]:
let rec print_levi2 obrnjen_trim_levi_cl =
  match obrnjen_trim_levi_cl with
  | [] -> ()
  | x :: xs -> Printf.printf "%c" x ; print_levi2 xs

val print_levi2 : char list -> unit = <fun>


In [57]:
let rec print_char_list = function 
  | [] -> ()
  | c :: cs -> Printf.printf "%c" c ; print_char_list cs

val print_char_list : char list -> unit = <fun>


In [58]:
let print_levi levi_char_list = 
  let obrni_ltrim_prestej_levi levi_char_list = 
    let rec aux_ltrim_in_prestej i flag acc sez =
      match flag, sez with
      | _, [] -> i, List.rev acc
      | false, x :: xs when x = ' ' -> aux_ltrim_in_prestej 0 false [] xs
      | false, x :: xs -> aux_ltrim_in_prestej (i + 1) true (x :: acc) xs
      | true, x :: xs -> aux_ltrim_in_prestej (i + 1) true (x :: acc) xs
    in
    List.rev levi_char_list 
    |> aux_ltrim_in_prestej 0 false []
  in
  let indeks, levi = obrni_ltrim_prestej_levi levi_char_list in
  print_char_list levi
  (* Aha sej bom mogla dat te pomožne v cel print, da bo tale indeks ostal nekje.*)

val print_levi : char list -> unit = <fun>


In [81]:
(* Še sprintat morm *)
let indeks5 = 5 in
let printanje_druge_vrste indeks =  (*dela tud za indeks = 0, pol nikol pač ne nardi*)
  Printf.printf "%s" "\n";
  for i=1 to indeks do
    Printf.printf "%c" ' '
  done;
  Printf.printf "%s" "^\n"
  (* To ne vem, zakaj, ne dela. Dela v terminalu ??*)

error: compile_error

In [ ]:
# let f elem =
  Printf.printf "I'm looking at element %d now\n" elem
in
  List.iter f my_list;;
I'm looking at element 1 now
I'm looking at element 2 now
I'm looking at element 3 now
I'm looking at element 4 now
I'm looking at element 5 now
I'm looking at element 6 now
I'm looking at element 7 now
I'm looking at element 8 now
I'm looking at element 9 now
I'm looking at element 10 now


In [84]:
let print_desni2 desni_char_list =
  let rec aux_printd_in_iskanje_konca stevec_presledkov sez = (* Sej v pomožnem so samo presledki!*)
    match sez with
    | [] -> ()
    | x :: xs when x = ' ' -> aux_printd_in_iskanje_konca (stevec_presledkov + 1) xs (* Če je presledek, prešteje, kok jih mora natisnt. *)
    (* To ma bs časovno zahtevnost, naredi raje seznam, al kaj. *)
    | x :: xs -> (
      for i=1 to stevec_presledkov do
        Printf.printf "%c" ' '
      done;
      Printf.printf "%c" x ; 
      aux_printd_in_iskanje_konca 0 xs (*Torej znak, če smo vmes shranjeval presledke, jih zdele še naprintamo*)
    )
  in
  aux_printd_in_iskanje_konca 0 desni_char_list

val print_desni2 : char list -> unit = <fun>


In [90]:
let print_triple = function
| (levi, glava, desni) -> (
  let obrni_ltrim_prestej_levi levi_char_list = 
    let rec aux_ltrim_in_prestej i flag acc sez =
      match flag, sez with
      | _, [] -> i, List.rev acc
      | false, x :: xs when x = ' ' -> aux_ltrim_in_prestej 0 false [] xs
      | false, x :: xs -> aux_ltrim_in_prestej (i + 1) true (x :: acc) xs
      | true, x :: xs -> aux_ltrim_in_prestej (i + 1) true (x :: acc) xs
    in
    List.rev levi_char_list 
    |> aux_ltrim_in_prestej 0 false []
  in 
  (* Vrne levi seznam pripravljen za print in njegovo dolžino. *)
  let rec print_char_list = function 
    | [] -> ()
    | c :: cs -> Printf.printf "%c" c ; print_char_list cs
  in
  let dolzina, levi_za_print = obrni_ltrim_prestej_levi levi in
  print_char_list levi_za_print; 
  (* Sprintan je levi del traku. *)
  Printf.printf "%c" glava;
  (* Sprintana je glava. *)
  let print_desni desni_char_list =
    let rec aux_printd_in_iskanje_konca stevec_presledkov sez = (* Sej v pomožnem so samo presledki!*)
      match sez with
      | [] -> ()
      | x :: xs when x = ' ' -> aux_printd_in_iskanje_konca (stevec_presledkov + 1) xs (* Če je presledek, prešteje, kok jih mora natisnt. *)
      (* To ma bs časovno zahtevnost, naredi raje seznam, al kaj. *)
      | x :: xs -> (
        for i=1 to stevec_presledkov do
          Printf.printf "%c" ' '
        done;
        Printf.printf "%c" x ; 
        aux_printd_in_iskanje_konca 0 xs (*Torej znak, če smo vmes shranjeval presledke, jih zdele še naprintamo*)
      )
    in
    aux_printd_in_iskanje_konca 0 desni_char_list
  in
  print_desni desni;
  (* Sprintan je desni del traku. *)
  Printf.printf "%s" "\n";
  for i=1 to dolzina do
    Printf.printf "%c" ' '
  done;
  Printf.printf "%s" "^\n"
  (* Sprintana je druga vrsta. *)
)

val print_triple : char list * char * char list -> unit = <fun>


In [91]:
let triple1 = (cl1, ' ', cl1)

val triple1 : char list * char * char list =
  ([' '; ' '; 'a'; 'b'; ' '; 'c'; ' '], ' ',
   [' '; ' '; 'a'; 'b'; ' '; 'c'; ' '])


In [92]:
print_triple triple1

- : unit = ()


In [93]:
type direction = Left | Right

type direction = Left | Right


In [94]:
let move dir trak = 
  match dir, trak with
  | Left, Trak (levi, glava, desni) -> (
    match levi with
    | [] -> Trak ([], ' ', glava :: desni)
    | x :: xs -> Trak (xs, x, glava :: desni) (* Je to vse ?*)
  )
  | Right, Trak (levi, glava, desni) -> (
    match desni with
    | [] -> Trak (glava :: levi, ' ', [])
    | x :: xs -> Trak (glava :: levi, x, xs)
  )

error: compile_error

In [ ]:
(* PRED ZAPISNIM TIPOM *)
module Tape : TAPE = struct
  type t = Trak of char list * char * char list
  (* Če sem prav razumela, je zapisni tip itak immutable in je vseen.
  Razmisli, a vseeno zamenjam na zapisni tip, če bodo zaradi tega lepše funkcije.
  POPRAVI !*)

  (*let char_list_to_string cl = String.concat "" (List.map (String.make 1) cl)*)
  (*Če bom še rabla to funkcijo, sem dala ven*)
  (* Pozab, to ma neko biggest loser slovenia casovno zahtevnost pomoje.*)
  let string_to_char_list niz = List.init (String.length niz) (String.get niz)
  (*Če bom rabla, dam ven iz make, pa mam kle*)
  (* A raj to nardim na roke, da je zihr O(n), sam sej te implementirane menda niso glupe?*)

  let make niz =
    let char_list = List.init (String.length niz) (String.get niz) in
    match char_list with
    | [] -> Trak ([], ' ', [])
    | x :: xs -> Trak ([], x, xs)

  let print = function
    | Trak (levi, glava, desni) -> (
      let obrni_ltrim_prestej_levi levi_char_list = 
        let rec aux_ltrim_in_prestej i flag acc sez =
          match flag, sez with
          | _, [] -> i, List.rev acc
          | false, x :: xs when x = ' ' -> aux_ltrim_in_prestej 0 false [] xs
          | false, x :: xs -> aux_ltrim_in_prestej (i + 1) true (x :: acc) xs
          | true, x :: xs -> aux_ltrim_in_prestej (i + 1) true (x :: acc) xs
        in
        List.rev levi_char_list 
        |> aux_ltrim_in_prestej 0 false []
      in 
      (* Vrne levi seznam pripravljen za print in njegovo dolžino. *)
      let rec print_char_list = function 
        | [] -> ()
        | c :: cs -> Printf.printf "%c" c ; print_char_list cs
      in
      let dolzina, levi_za_print = obrni_ltrim_prestej_levi levi in
      print_char_list levi_za_print; 
      (* Sprintan je levi del traku. *)
      Printf.printf "%c" glava;
      (* Sprintana je glava. *)
      let print_desni desni_char_list =
        let rec aux_printd_in_iskanje_konca stevec_presledkov sez = (* Sej v pomožnem so samo presledki!*)
          match sez with
          | [] -> ()
          | x :: xs when x = ' ' -> aux_printd_in_iskanje_konca (stevec_presledkov + 1) xs (* Če je presledek, prešteje, kok jih mora natisnt. *)
          (* To ma bs časovno zahtevnost, naredi raje seznam, al kaj. *)
          | x :: xs -> (
            for i=1 to stevec_presledkov do
              Printf.printf "%c" ' '
            done;
            Printf.printf "%c" x ; 
            aux_printd_in_iskanje_konca 0 xs (*Torej znak, če smo vmes shranjeval presledke, jih zdele še naprintamo*)
          )
        in
        aux_printd_in_iskanje_konca 0 desni_char_list
      in
      print_desni desni;
      (* Sprintan je desni del traku. *)
      Printf.printf "%s" "\n";
      for i=1 to dolzina do
        Printf.printf "%c" ' '
      done;
      Printf.printf "%s" "^\n"
      (* Sprintana je druga vrsta. *)
    )

  let read = function
    | Trak (_, glava, _) -> glava
  let move dir trak = 
    match dir, trak with
    | Left, Trak (levi, glava, desni) -> (
      match levi with
      | [] -> Trak ([], ' ', glava :: desni)
      | x :: xs -> Trak (xs, x, glava :: desni)
    )
    | Right, Trak (levi, glava, desni) -> (
      match desni with
      | [] -> Trak (glava :: levi, ' ', [])
      | x :: xs -> Trak (glava :: levi, x, xs)
    )
  let write c = function
    | Trak (levi, glava, desni) -> Trak (levi, c, desni)
end

let primer_trak = Tape.(
  make "ABCDE"
  |> move Left
  |> move Left
  |> move Right
  |> move Right
  |> move Right
  |> move Right
  |> write '!'
  |> print
)